# Sustainability Regression

**Nations analyzed:**
1. Germany `DEU`
2. Brazil `BRA`
3. Chile `CHL`
4. Switzerland `CHE`
5. Czech Republic (Czechia) `CZE`
6. Denmark `DNK`


# Introduction:

In this project, we will work on an exciting sustainability problem with many aspects of data science and business decision modeling that we will see in real-world situations. We are going to use real-world data from the World Development Indicators and Climate Change Knowledge Portal;


# Set Up Environment and Data



## Import Python Libraries

Set up the environment with modules for data manipulation and analysis:
* Pandas, Numpy, etc.
* Solver, mixed-integer nonlinear: `bonmin`

In [ ]:
# import modules
import pandas as pd
import numpy as np
from pylab import * 
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

# import and set up required solver, bonmin
if not (shutil.which("bonmin") or os.path.isfile("bonmin")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
        !unzip -o -q bonmin-linux64
        # !apt-get install -y -qq mindtpy
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("bonmin") or os.path.isfile("bonmin"))

from pyomo.environ import *

     |████████████████████████████████| 9.2 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 3.5 MB/s 


## Retreive and Read the Data
First, we download the raw data `Data` and data dictionary `Series` from the Excel worksheet using `pd.read_excel()`.

* Read the raw data on the 'Data' tab as `df_data`
* Read the data dictionary `Series` tab as `df_series`

We used `!gdown -- id SHARELINK` to retreive dataset in Google Drive

In [ ]:
# https://docs.google.com/spreadsheets/d/1cB3ep8hBXrUcBp-DHoplEpJABaZsaCQ3/edit?usp=sharing&ouid=105133078201960946471&rtpof=true&sd=true
!gdown --id 1cB3ep8hBXrUcBp-DHoplEpJABaZsaCQ3
df_data = pd.read_excel('climate_change_download_0.xls',sheet_name='Data') # create the df_data DataFrame
df_series = pd.read_excel('climate_change_download_0.xls',sheet_name='Series') # create the df_series DataFrame

Downloading...
From: https://drive.google.com/uc?id=1cB3ep8hBXrUcBp-DHoplEpJABaZsaCQ3
To: /content/climate_change_download_0.xls
100% 4.86M/4.86M [00:00<00:00, 42.8MB/s]


Read and explore `df_data`. We see that there are country codes, country names matched to the codes, a series code and connected name, and a number of numeric year columns with lots of missing data.

In [ ]:
df_data.head()

,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.5748,..,..,..,..,..,..,..,..,..,29.5748,..,..,..,..,..,..,..,..,..,..,..
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,..,..,..,..,..,..,0.208235,..,..,..,..,..,..,..,..,..,..,..
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.96788,..,..,..,..,..,..,..,..,..,4.96788,..,..,..,..,..,..,..,..,..,..,..


Read the data dictionary dataset `df_series`. We see the same series codes as those from the `df_data` DataFrame. This DataFrame introduces new information about specific topics that series are related to and where data is sourced from.

In [ ]:
df_series.head()

,Series code,Series name,Scale,Decimals,Order,Topic,Definition,Source
0,SP.POP.TOTL,Population,0,0,1,Size of the economy,Population includes all residents who are pres...,(1) United Nations Population Division. 2011. ...
1,SP.POP.GROW,Population growth (annual %),0,1,2,Size of the economy,Annual population growth rate for year t is th...,Derived from total population. Population sour...
2,NY.GDP.MKTP.CD,GDP ($),0,0,3,Size of the economy,GDP is gross domestic product and measures the...,"World Bank national accounts data, and OECD Na..."
3,NY.GNP.PCAP.CD,GNI per capita (Atlas $),0,0,4,Size of the economy,"GNI per capita is the gross national income, c...","World Bank national accounts data, and OECD Na..."
4,EN.CLC.MMDT.C,"Average daily min/max temperature (1961-1990, ...",Text,Text,5,Climate,Average daily min/max temperature are the mini...,"Mitchell, T.D., Carter, T.R., Jones, P.D., Hul..."


## Merge the Dataframes

Join the `df_data` and `df_series` datasets together as `df`. This gives us all of the required data in one convenient place.
* Use the common column `Series code` as the column to join on.

In [ ]:
df = df_data.merge(df_series, left_on='Series code', right_on='Series code') # create the df with a merge on Series Code

# check the work
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13512 entries, 0 to 13511
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Country code   13512 non-null  object
 1   Country name   13512 non-null  object
 2   Series code    13512 non-null  object
 3   Series name_x  13512 non-null  object
 4   SCALE          13512 non-null  object
 5   Decimals_x     13512 non-null  object
 6   1990           10017 non-null  object
 7   1991           10017 non-null  object
 8   1992           10017 non-null  object
 9   1993           10017 non-null  object
 10  1994           10017 non-null  object
 11  1995           10017 non-null  object
 12  1996           10017 non-null  object
 13  1997           10017 non-null  object
 14  1998           10017 non-null  object
 15  1999           10017 non-null  object
 16  2000           10017 non-null  object
 17  2001           10017 non-null  object
 18  2002           10017 non-n

,Country code,Country name,Series code,Series name_x,SCALE,Decimals_x,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,Series name_y,Scale,Decimals_y,Order,Topic,Definition,Source
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.5748,..,..,..,..,..,..,..,..,..,29.5748,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,..,..,..,..,..,..,0.208235,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.96788,..,..,..,..,..,..,..,..,..,4.96788,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...


# Clean the 'df' DataFrame

Now that our data is in one location, we can eliminate redundancies and start making the data easier to work with.
* Delete the redundant data columns
* Rename columns to fit for better coding

In [ ]:
# first, drop any columns that have 100% duplicate information
del df['Series name_y']
del df['Decimals_y']

# rename 'Series name_x' to just 'Series name'
df.rename(columns={'Series name_x':'Series name'}, inplace=True)
# rename 'Decimals_x' to just 'Decimals'
df.rename(columns={'Decimals_x':'Decimals'}, inplace=True)
df # display output to validate results

,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,Scale,Order,Topic,Definition,Source
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.5748,..,..,..,..,..,..,..,..,..,29.5748,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,..,..,..,..,..,..,0.208235,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.96788,..,..,..,..,..,..,..,..,..,4.96788,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13507,YEM,"Yemen, Rep.",SP.URB.TOTL,Urban population,0,0,2.49718e+06,2.69364e+06,2.90976e+06,3.13964e+06,3.37393e+06,3.60526e+06,3.81758e+06,4.02428e+06,4.22936e+06,4.43979e+06,4.6612e+06,4.89894e+06,5.14862e+06,5.41033e+06,5.68341e+06,5.96746e+06,6.27572e+06,6.59727e+06,6.93279e+06,7.28307e+06,7.6487e+06,..,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...
13508,ZAF,South Africa,SP.URB.TOTL,Urban population,0,0,18304000,1.88649e+07,1.94461e+07,2.00485e+07,2.06729e+07,21320400,2.19921e+07,2.26976e+07,2.34387e+07,2.42174e+07,25036000,2.57692e+07,2.63456e+07,2.69044e+07,2.74482e+07,2.79887e+07,2.85336e+07,2.90798e+07,2.96369e+07,3.01938e+07,3.08446e+07,..,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...
13509,ZAR,"Congo, Dem. Rep.",SP.URB.TOTL,Urban population,0,0,1.01209e+07,1.05695e+07,1.106e+07,1.15686e+07,1.20615e+07,1.25151e+07,1.29906e+07,1.34286e+07,1.3852e+07,1.42965e+07,1.47886e+07,1.54294e+07,1.61253e+07,1.68678e+07,1.76408e+07,1.8432e+07,1.93337e+07,2.02614e+07,2.12165e+07,2.22018e+07,2.322e+07,..,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...
13510,ZMB,Zambia,SP.URB.TOTL,Urban population,0,0,3.09686e+06,3.14167e+06,3.18326e+06,3.22351e+06,3.26494e+06,3.30912e+06,3.35693e+06,3.40748e+06,3.45843e+06,3.50665e+06,3.55014e+06,3.64072e+06,3.72988e+06,3.81964e+06,3.91287e+06,4.01183e+06,4.12899e+06,4.25314e+06,4.38486e+06,4.52456e+06,4.61473e+06,..,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...


## Reduce Annual Data To Averaged Values

The datasets cover multiple years. Individual year data is not required to satisfy this analysis. Consolidating this data entails:`
1. Subset the year data columns into a temporary dataframe
2. Investigate data type and update to numeric type
3. Create a column averaging each row



**Subset**

columns for the years, 1990 - 2011.

In [ ]:
# subset the years by iloc
years = df.iloc[:,6:28] # the df columns with year data
years # display to check work

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,29.5748,..,..,..,..,..,..,..,..,..,29.5748,..,..,..,..,..,..,..,..,..,..,..
1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
2,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
3,0.208235,..,..,..,..,..,..,..,..,..,0.208235,..,..,..,..,..,..,..,..,..,..,..
4,4.96788,..,..,..,..,..,..,..,..,..,4.96788,..,..,..,..,..,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13507,2.49718e+06,2.69364e+06,2.90976e+06,3.13964e+06,3.37393e+06,3.60526e+06,3.81758e+06,4.02428e+06,4.22936e+06,4.43979e+06,4.6612e+06,4.89894e+06,5.14862e+06,5.41033e+06,5.68341e+06,5.96746e+06,6.27572e+06,6.59727e+06,6.93279e+06,7.28307e+06,7.6487e+06,..
13508,18304000,1.88649e+07,1.94461e+07,2.00485e+07,2.06729e+07,21320400,2.19921e+07,2.26976e+07,2.34387e+07,2.42174e+07,25036000,2.57692e+07,2.63456e+07,2.69044e+07,2.74482e+07,2.79887e+07,2.85336e+07,2.90798e+07,2.96369e+07,3.01938e+07,3.08446e+07,..
13509,1.01209e+07,1.05695e+07,1.106e+07,1.15686e+07,1.20615e+07,1.25151e+07,1.29906e+07,1.34286e+07,1.3852e+07,1.42965e+07,1.47886e+07,1.54294e+07,1.61253e+07,1.68678e+07,1.76408e+07,1.8432e+07,1.93337e+07,2.02614e+07,2.12165e+07,2.22018e+07,2.322e+07,..
13510,3.09686e+06,3.14167e+06,3.18326e+06,3.22351e+06,3.26494e+06,3.30912e+06,3.35693e+06,3.40748e+06,3.45843e+06,3.50665e+06,3.55014e+06,3.64072e+06,3.72988e+06,3.81964e+06,3.91287e+06,4.01183e+06,4.12899e+06,4.25314e+06,4.38486e+06,4.52456e+06,4.61473e+06,..


**Data type** 

Check data types and turn objects to numeric as required

In [ ]:
#Print info on  data types and the shape of the data being used
print(years.info())
print(years.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13512 entries, 0 to 13511
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1990    10017 non-null  object
 1   1991    10017 non-null  object
 2   1992    10017 non-null  object
 3   1993    10017 non-null  object
 4   1994    10017 non-null  object
 5   1995    10017 non-null  object
 6   1996    10017 non-null  object
 7   1997    10017 non-null  object
 8   1998    10017 non-null  object
 9   1999    10017 non-null  object
 10  2000    10017 non-null  object
 11  2001    10017 non-null  object
 12  2002    10017 non-null  object
 13  2003    10017 non-null  object
 14  2004    10017 non-null  object
 15  2005    10017 non-null  object
 16  2006    10017 non-null  object
 17  2007    10017 non-null  object
 18  2008    10017 non-null  object
 19  2009    10017 non-null  object
 20  2010    10017 non-null  object
 21  2011    12382 non-null  object
dtypes: object(22)
memory u

In [ ]:
# convert the dataframe to numeric
years = years.apply(pd.to_numeric, errors = 'coerce')
years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13512 entries, 0 to 13511
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1990    4854 non-null   float64
 1   1991    3497 non-null   float64
 2   1992    3653 non-null   float64
 3   1993    3717 non-null   float64
 4   1994    3779 non-null   float64
 5   1995    4672 non-null   float64
 6   1996    3804 non-null   float64
 7   1997    3767 non-null   float64
 8   1998    3818 non-null   float64
 9   1999    4005 non-null   float64
 10  2000    5496 non-null   float64
 11  2001    4018 non-null   float64
 12  2002    4057 non-null   float64
 13  2003    4043 non-null   float64
 14  2004    4225 non-null   float64
 15  2005    5084 non-null   float64
 16  2006    4236 non-null   float64
 17  2007    4248 non-null   float64
 18  2008    4603 non-null   float64
 19  2009    3761 non-null   float64
 20  2010    2332 non-null   float64
 21  2011    658 non-null    float64
dty

**Average the year data into `df`** 

Use the resultant numeric year data to create a column of averages in `df`



In [ ]:
# add the mean of columns to df in a new column 'average'
df['average'] = years.mean(axis = 1)
df.head() # check work

,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,Scale,Order,Topic,Definition,Source,average
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.5748,..,..,..,..,..,..,..,..,..,29.5748,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,29.574810
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,..,..,..,..,..,..,0.208235,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.208235
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.96788,..,..,..,..,..,..,..,..,..,4.96788,..,..,..,..,..,..,..,..,..,..,..,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,4.967875


**Simplify** 

Remove the unecessary year columns. We do not need individual year data for this analysis. The new `average` column will suffice.

In [ ]:
# drop the individual years for each variable
df = df.drop(years, axis = 1) # remove the columns that we already subsected as 'years'
df.info() # check result

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13512 entries, 0 to 13511
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country code  13512 non-null  object 
 1   Country name  13512 non-null  object 
 2   Series code   13512 non-null  object 
 3   Series name   13512 non-null  object 
 4   SCALE         13512 non-null  object 
 5   Decimals      13512 non-null  object 
 6   Scale         13512 non-null  object 
 7   Order         13512 non-null  int64  
 8   Topic         13512 non-null  object 
 9   Definition    13512 non-null  object 
 10  Source        13512 non-null  object 
 11  average       7891 non-null   float64
dtypes: float64(1), int64(1), object(10)
memory usage: 1.3+ MB


## Drop missing values

Remove the NaNs or non-null values. Some have appeared in the `average` column. See output count of `df.info` visible above to confirm that there are missing values.

Use `df.dropna`

In [ ]:
# use dropna fuction to drop the missing value
df.dropna(inplace = True)
# check the work
print(df.shape)

(7891, 12)


All students should have 7891 rows at this point! And guess what? We do! 👍

# Filter `df` Down to Required Countries for Analysis

Reduce `df` to only the country data required using the `df.isin()` command.
* set `my_countries` to the six countries required

|Code|Country|
|:--|:--|
|BRA|Brazil|
|CHE|Switzerland|
|CHL|Chile|
|CZE| Czechia / Czech Republic|
|DEU| Germany|
|DMK|Denmark|

In [ ]:
# Germany, Brazil, Chile, Switzerland, Czechia, Denmark
my_countries = ['DEU','BRA','CHL','CHE','CZE','DNK']
df = df[df['Country code'].isin(my_countries)] 
df  ## 229 rows × 12 columns

,Country code,Country name,Series code,Series name,SCALE,Decimals,Scale,Order,Topic,Definition,Source,average
26,BRA,Brazil,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,1.212341e+00
33,CHE,Switzerland,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000e+00
35,CHL,Chile,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,3.080679e+00
49,CZE,Czech Republic,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000e+00
50,DEU,Germany,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,4.917982e+00
...,...,...,...,...,...,...,...,...,...,...,...,...
13312,CHE,Switzerland,SP.URB.TOTL,Urban population,0,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,5.308964e+06
13314,CHL,Chile,SP.URB.TOTL,Urban population,0,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,1.319908e+07
13328,CZE,Czech Republic,SP.URB.TOTL,Urban population,0,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,7.644853e+06
13329,DEU,Germany,SP.URB.TOTL,Urban population,0,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,5.996282e+07


In [ ]:
# check the target variables
df[df['Series name'] == 'CO2 emissions, total (KtCO2)']

,Country code,Country name,Series code,Series name,SCALE,Decimals,Scale,Order,Topic,Definition,Source,average
1657,BRA,Brazil,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",304796.215000
1664,CHE,Switzerland,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",41050.135000
1666,CHL,Chile,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",53241.752000
1680,CZE,Czech Republic,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",125148.885941
1681,DEU,Germany,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",817294.591000
1684,DNK,Denmark,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",53926.323000


# Filter Series to Only Those Present for All Countries

Not all data exists for all countries. The variables must be common to all countries, so we find incomplete data through a simple check of the count of data appearance for each series indicator. A count of 6 means that data is complete (present for all countries) for that indicator.

In [ ]:
# group the variables by Series name and count the appears number
df.groupby(['Series name']).count()

,Country code,Country name,Series code,SCALE,Decimals,Scale,Order,Topic,Definition,Source,average
Series name,,,,,,,,,,,
Access to electricity (% of total population),2,2,2,2,2,2,2,2,2,2,2
Access to improved sanitation (% of total pop.),6,6,6,6,6,6,6,6,6,6,6
Access to improved water source (% of total pop.),6,6,6,6,6,6,6,6,6,6,6
Agricultural land under irrigation (% of total ag. land),3,3,3,3,3,3,3,3,3,3,3
Annual freshwater withdrawals (% of internal resources),6,6,6,6,6,6,6,6,6,6,6
"Average annual precipitation (1961-1990, mm)",6,6,6,6,6,6,6,6,6,6,6
CO2 emissions per capita (metric tons),6,6,6,6,6,6,6,6,6,6,6
"CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)",6,6,6,6,6,6,6,6,6,6,6
"CO2 emissions, total (KtCO2)",6,6,6,6,6,6,6,6,6,6,6


Only data that matches for the count of countries (6) is complete and can be used. Create a DataFrame for those counts to be used in reducing the dataset to viable data only.

In [ ]:
# Create a DataFrame for the count of countries with data for each indicator
count_value_df = df.groupby(['Series name']).count()
count_value_df # check work

,Country code,Country name,Series code,SCALE,Decimals,Scale,Order,Topic,Definition,Source,average
Series name,,,,,,,,,,,
Access to electricity (% of total population),2,2,2,2,2,2,2,2,2,2,2
Access to improved sanitation (% of total pop.),6,6,6,6,6,6,6,6,6,6,6
Access to improved water source (% of total pop.),6,6,6,6,6,6,6,6,6,6,6
Agricultural land under irrigation (% of total ag. land),3,3,3,3,3,3,3,3,3,3,3
Annual freshwater withdrawals (% of internal resources),6,6,6,6,6,6,6,6,6,6,6
"Average annual precipitation (1961-1990, mm)",6,6,6,6,6,6,6,6,6,6,6
CO2 emissions per capita (metric tons),6,6,6,6,6,6,6,6,6,6,6
"CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)",6,6,6,6,6,6,6,6,6,6,6
"CO2 emissions, total (KtCO2)",6,6,6,6,6,6,6,6,6,6,6


Return the variables where there are at least 6 rows. 33 rows meet the criteria.

In [ ]:
# show the variables has 5 rows
count_value_df = count_value_df[count_value_df['average'] == 6]
print(count_value_df)
print(count_value_df.shape)

                                                    Country code  ...  average
Series name                                                       ...         
Access to improved sanitation (% of total pop.)                6  ...        6
Access to improved water source (% of total pop.)              6  ...        6
Annual freshwater withdrawals (% of internal re...             6  ...        6
Average annual precipitation (1961-1990, mm)                   6  ...        6
CO2 emissions per capita (metric tons)                         6  ...        6
CO2 emissions per units of GDP (kg/$1,000 of 20...             6  ...        6
CO2 emissions, total (KtCO2)                                   6  ...        6
Cereal yield (kg per hectare)                                  6  ...        6
Droughts, floods, extreme temps (% pop. avg. 19...             6  ...        6
Ease of doing business (ranking 1-183; 1=best)                 6  ...        6
Energy use per capita (kilograms of oil equival...  

Create a new variable called `shared_list` which is the index of `count_value_df` (variables which satisfy the criteria for completeness).

In [ ]:
# subset the Series name_x
shared_list = count_value_df.index
shared_list

Index(['Access to improved sanitation (% of total pop.)',
       'Access to improved water source (% of total pop.)',
       'Annual freshwater withdrawals (% of internal resources)',
       'Average annual precipitation (1961-1990, mm)',
       'CO2 emissions per capita (metric tons)',
       'CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)',
       'CO2 emissions, total (KtCO2)', 'Cereal yield (kg per hectare)',
       'Droughts, floods, extreme temps (% pop. avg. 1990-2009)',
       'Ease of doing business (ranking 1-183; 1=best)',
       'Energy use per capita (kilograms of oil equivalent)',
       'Energy use per units of GDP (kg oil eq./$1,000 of 2005 PPP $)',
       'Foreign direct investment, net inflows (% of GDP)', 'GDP ($)',
       'GHG net emissions/removals by LUCF (MtCO2e)',
       'GNI per capita (Atlas $)', 'Land area below 5m (% of land area)',
       'Methane (CH4) emissions, total (KtCO2e)',
       'Nationally terrestrial protected areas (% of total land are

Subset the rows from `df` that are `isin()` the shared list. 198 rows result.

In [ ]:
# subset the rows from df that are isin() the shared list
df = df[df['Series name'].isin(shared_list)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198 entries, 26 to 13332
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country code  198 non-null    object 
 1   Country name  198 non-null    object 
 2   Series code   198 non-null    object 
 3   Series name   198 non-null    object 
 4   SCALE         198 non-null    object 
 5   Decimals      198 non-null    object 
 6   Scale         198 non-null    object 
 7   Order         198 non-null    int64  
 8   Topic         198 non-null    object 
 9   Definition    198 non-null    object 
 10  Source        198 non-null    object 
 11  average       198 non-null    float64
dtypes: float64(1), int64(1), object(10)
memory usage: 20.1+ KB


Topics are critical to analysis, as only one series per topic will be considered. Retreive the unique topics in `df`

In [ ]:
# See the topics
df['Topic'].unique()

array(['Exposure to impacts', 'Resilience',
       'GHG emissions and energy use', 'Climate', 'Size of the economy'],
      dtype=object)

# Extract Required Parameters from `df`

`df` data (cleaned and filtered) must now be broken down for use in the Solver model.


## Subset rows that fit requirements

Analysis requires not having the Topic == GHG emissions and energy use

Make a copy of `df` as `tmp`, and subset rows where topic does not match 'GHG emissions and energy use'

In [ ]:
tmp = df                                                  #Create copy dataframe
tmp = tmp[tmp['Topic']!='GHG emissions and energy use']   #Drop rows regarding GHG Emissions
tmp.shape                                                 #Print the shape to see how many entries are present

(144, 12)

## Use the temp dataset to create a dictionary for variables


Zip together the country codes, series, and average from the temp DataFrame as `var_dict`. 

In [ ]:
# make the key and the value of each key then convert all to dictionary
var_dict = dict(zip(zip(tmp['Country code'], tmp['Series code']), tmp['average']))
# check the dictionary
var_dict

{('BRA', 'AG.LND.EL5M.ZS'): 1.212341,
 ('BRA', 'AG.YLD.CREL.KG'): 2774.025,
 ('BRA', 'BX.KLT.DINV.WD.GD.ZS'): 2.1135599986872826,
 ('BRA', 'EN.CLC.HPPT.MM'): 1782.0,
 ('BRA', 'EN.CLC.MDAT.ZS'): 0.4824510279925466,
 ('BRA', 'EN.POP.EL5M.ZS'): 4.8121455,
 ('BRA', 'EN.URB.MCTY.TL.ZS'): 37.39836944270659,
 ('BRA', 'ER.H2O.FWTL.ZS'): 1.0596776178171565,
 ('BRA', 'ER.LND.PTLD.ZS'): 18.905910408695,
 ('BRA', 'IC.BUS.EASE.XQ'): 125.66666666666667,
 ('BRA', 'IS.ROD.PAVE.ZS'): 8.6636365110224,
 ('BRA', 'NY.GDP.MKTP.CD'): 845035445998.8274,
 ('BRA', 'NY.GNP.PCAP.CD'): 4412.857142857143,
 ('BRA', 'SE.ENR.PRSC.FM.ZS'): 102.82137499999999,
 ('BRA', 'SE.PRM.CMPT.ZS'): 102.52362333333333,
 ('BRA', 'SH.DYN.MORT'): 37.30952380952381,
 ('BRA', 'SH.H2O.SAFE.ZS'): 92.8,
 ('BRA', 'SH.MED.NUMW.P3'): 4.416333333333333,
 ('BRA', 'SH.MED.PHYS.ZS'): 1.4012500017875003,
 ('BRA', 'SH.STA.ACSN'): 74.8,
 ('BRA', 'SP.POP.GROW'): 1.3413778964764749,
 ('BRA', 'SP.POP.TOTL'): 173654627.52380952,
 ('BRA', 'SP.URB.GROW'):

Create three new variables that have unique values:
* `countries` (for each unique country in the analysis)
* `indicators` (as the series that fit all out filtering parameters above)
* `mygroups` (as the topics that fit the filtering parameters above)

In [ ]:
# make the list will be used in the model
countries = df['Country code'].unique() ## the team countries
indicators = df['Series code'].unique() ## the variables will be used in model
mygroups = df['Topic'].unique()         ## categories of variables
# Check the countries
print(countries)
# Check the variables
print(indicators)
# Check the categories
print(mygroups)

['BRA' 'CHE' 'CHL' 'CZE' 'DEU' 'DNK']
['AG.LND.EL5M.ZS' 'AG.YLD.CREL.KG' 'BX.KLT.DINV.WD.GD.ZS'
 'EG.USE.COMM.GD.PP.KD' 'EG.USE.PCAP.KG.OE' 'EN.ATM.CO2E.KT'
 'EN.ATM.CO2E.PC' 'EN.ATM.CO2E.PP.GD.KD' 'EN.ATM.GHGO.KT.CE'
 'EN.ATM.METH.KT.CE' 'EN.ATM.NOXE.KT.CE' 'EN.CLC.GHGR.MT.CE'
 'EN.CLC.HPPT.MM' 'EN.CLC.MDAT.ZS' 'EN.POP.EL5M.ZS' 'EN.URB.MCTY.TL.ZS'
 'ER.H2O.FWTL.ZS' 'ER.LND.PTLD.ZS' 'IC.BUS.EASE.XQ' 'IS.ROD.PAVE.ZS'
 'NY.GDP.MKTP.CD' 'NY.GNP.PCAP.CD' 'SE.ENR.PRSC.FM.ZS' 'SE.PRM.CMPT.ZS'
 'SH.DYN.MORT' 'SH.H2O.SAFE.ZS' 'SH.MED.NUMW.P3' 'SH.MED.PHYS.ZS'
 'SH.STA.ACSN' 'SP.POP.GROW' 'SP.POP.TOTL' 'SP.URB.GROW' 'SP.URB.TOTL']
['Exposure to impacts' 'Resilience' 'GHG emissions and energy use'
 'Climate' 'Size of the economy']


We will **NOT** use variables from 'GHG emissions and energy use' to predict carbon dioxide emissions `"EN.ATM.CO2E.KT"` .

For example, we will just use variables like 'Exposure to impacts', 'Resilience' and 'Size of the economy' to predict the greenhouse gas variable `"EN.ATM.CO2E.KT"` 

# Extract list of series per Topic



Create a variable `topics` that contains all of the topic groups except for the unwanted `'GHG emissions and energy use'`.

In [ ]:
# check if the topic is not from GHG emissions
topics = list(df['Topic'].unique())
topics.remove('GHG emissions and energy use')
print(topics)

['Exposure to impacts', 'Resilience', 'Climate', 'Size of the economy']


Create a variable for an empty dictionary `indicator_dict` This will contain a coerced one series per topic.

In [ ]:
# use topic as the key and add a list of the related variables
indicator_dict = dict()
for topic in topics:
  indicator_dict[topic] = list(df[df['Topic'] == topic]['Series code'].unique())

In [ ]:
indicator_dict

{'Climate': ['EN.CLC.HPPT.MM'],
 'Exposure to impacts': ['AG.LND.EL5M.ZS',
  'EN.CLC.MDAT.ZS',
  'EN.POP.EL5M.ZS',
  'EN.URB.MCTY.TL.ZS',
  'ER.H2O.FWTL.ZS',
  'ER.LND.PTLD.ZS',
  'SH.DYN.MORT',
  'SP.URB.GROW',
  'SP.URB.TOTL'],
 'Resilience': ['AG.YLD.CREL.KG',
  'BX.KLT.DINV.WD.GD.ZS',
  'IC.BUS.EASE.XQ',
  'IS.ROD.PAVE.ZS',
  'SE.ENR.PRSC.FM.ZS',
  'SE.PRM.CMPT.ZS',
  'SH.H2O.SAFE.ZS',
  'SH.MED.NUMW.P3',
  'SH.MED.PHYS.ZS',
  'SH.STA.ACSN'],
 'Size of the economy': ['NY.GDP.MKTP.CD',
  'NY.GNP.PCAP.CD',
  'SP.POP.GROW',
  'SP.POP.TOTL']}

Convert all dictionary values to a list as `indicators`. This list is key to modeling.

In [ ]:
# all values in indicator_dict
indicators = sorted({x for v in indicator_dict.values() for x in v})
indicators

['AG.LND.EL5M.ZS',
 'AG.YLD.CREL.KG',
 'BX.KLT.DINV.WD.GD.ZS',
 'EN.CLC.HPPT.MM',
 'EN.CLC.MDAT.ZS',
 'EN.POP.EL5M.ZS',
 'EN.URB.MCTY.TL.ZS',
 'ER.H2O.FWTL.ZS',
 'ER.LND.PTLD.ZS',
 'IC.BUS.EASE.XQ',
 'IS.ROD.PAVE.ZS',
 'NY.GDP.MKTP.CD',
 'NY.GNP.PCAP.CD',
 'SE.ENR.PRSC.FM.ZS',
 'SE.PRM.CMPT.ZS',
 'SH.DYN.MORT',
 'SH.H2O.SAFE.ZS',
 'SH.MED.NUMW.P3',
 'SH.MED.PHYS.ZS',
 'SH.STA.ACSN',
 'SP.POP.GROW',
 'SP.POP.TOTL',
 'SP.URB.GROW',
 'SP.URB.TOTL']

In [ ]:
#Show the data
df

,Country code,Country name,Series code,Series name,SCALE,Decimals,Scale,Order,Topic,Definition,Source,average
26,BRA,Brazil,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,1.212341e+00
33,CHE,Switzerland,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000e+00
35,CHL,Chile,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,3.080679e+00
49,CZE,Czech Republic,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000e+00
50,DEU,Germany,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,4.917982e+00
...,...,...,...,...,...,...,...,...,...,...,...,...
13312,CHE,Switzerland,SP.URB.TOTL,Urban population,0,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,5.308964e+06
13314,CHL,Chile,SP.URB.TOTL,Urban population,0,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,1.319908e+07
13328,CZE,Czech Republic,SP.URB.TOTL,Urban population,0,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,7.644853e+06
13329,DEU,Germany,SP.URB.TOTL,Urban population,0,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,5.996282e+07


# Model
Our desired model should function as though it has 6 rows of data (one for each country) and ~20 predictor variables. Obviously such a limited dataset requires some restriction (regularization) on the number of variables that can go into the model.

Per analysis requirements, we will only say that four variables can go into the model.

## Concrete Model

In [ ]:
# declare the model
model = ConcreteModel()

## Decision Variables
You should have multiple variables here
* `a` are the coefficients on the terms in your model - one `a` for each variable. Use `Var(indicators, domain=XYZ, bounds=ABC)` to do this.
* `activation` is whether or not the variable turns on (1) or not (0) - one `activation` for each variable in the model.  Use `Var(indicators, domain=XYZ, bounds=ABC)` to do this.
* `y` is a decision variable which enforces the linear form of the model (these are your intermediate predictions for each country, so you should only have 6).  Use `Var(countries, domain=XYZ, bounds=ABC, initialize=Z)` to do this.

In [ ]:
# declare the decision variables
model.a = Var(indicators, domain=Reals, bounds = (-100, 100)) # coefficients
model.y = Var(countries, domain=NonNegativeReals, initialize=0) # predict target variable
model.activation = Var(indicators, domain = Binary) # activation variables


## Constraints

In [ ]:
# Constraints
model.constraints = ConstraintList()

In [ ]:
# constraint for the topics
# declare variables
topic1=0
topic2=0
topic3=0
topic4=0


for indicator in indicator_dict[(list(indicator_dict.keys())[0])]:
  topic1 += model.activation[indicator]

for indicator in indicator_dict[(list(indicator_dict.keys())[1])]:
  topic2 += model.activation[indicator]

for indicator in indicator_dict[(list(indicator_dict.keys())[2])]:
  topic3 += model.activation[indicator]

for indicator in indicator_dict[(list(indicator_dict.keys())[3])]:
  topic4 += model.activation[indicator]

# constraint for the predict variables
# declare variables
Predict_BRA=0
Predict_CHE=0
Predict_CHL=0
Predict_CZE=0
Predict_DEU=0
Predict_DNK=0

# the equation for the predict variables
for indicator in indicators:
 Predict_BRA += model.a[indicator]*model.activation[indicator]*var_dict['BRA',indicator] # here is the perdicting formula y = A1N1 + A2N2 + .... AnNn

for indicator in indicators:
  Predict_CHE += model.a[indicator]*model.activation[indicator]*var_dict['CHE',indicator]

for indicator in indicators:
  Predict_CHL += model.a[indicator]*model.activation[indicator]*var_dict['CHL',indicator]

for indicator in indicators:
  Predict_CZE += model.a[indicator]*model.activation[indicator]*var_dict['CZE',indicator]

for indicator in indicators:
  Predict_DEU += model.a[indicator]*model.activation[indicator]*var_dict['DEU',indicator]

for indicator in indicators:
  Predict_DNK += model.a[indicator]*model.activation[indicator]*var_dict['DNK',indicator]

# add the constraints
model.constraints.add(topic1 == 1) # we should at least choose one indicator for each topic
model.constraints.add(topic2 == 1)
model.constraints.add(topic3 == 1)
model.constraints.add(topic4 == 1)

model.constraints.add(Predict_BRA == model.y['BRA']) # these are the equations to bulld linear regression models for each country
model.constraints.add(Predict_CHE == model.y['CHE'])
model.constraints.add(Predict_CHL == model.y['CHL'])
model.constraints.add(Predict_CZE == model.y['CZE'])
model.constraints.add(Predict_DEU == model.y['DEU'])
model.constraints.add(Predict_DNK == model.y['DNK'])

In [ ]:
# check the actual number of our target variable, we will need that in the model
df[df['Series name'] == 'CO2 emissions, total (KtCO2)']

,Country code,Country name,Series code,Series name,SCALE,Decimals,Scale,Order,Topic,Definition,Source,average
1657,BRA,Brazil,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",304796.215000
1664,CHE,Switzerland,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",41050.135000
1666,CHL,Chile,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",53241.752000
1680,CZE,Czech Republic,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",125148.885941
1681,DEU,Germany,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",817294.591000
1684,DNK,Denmark,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",53926.323000


##Objective Function

In [ ]:
# Objective
obj_expr = 0
for country in countries:
    obj_expr += (float(df['average'][(df['Series name'] == 'CO2 emissions, total (KtCO2)') & (df['Country code'] == country)]) - model.y[country]) ** 2
    

model.cost = Objective(
    expr = obj_expr, 
    sense = minimize) # we want the least sum of squares, so this is a minimization model

## Pretty print
Check our constraints. We are all system go! 👍

In [ ]:
model.pprint()
# check the result, we have a for each indicator, activation for each indicator
# objective function looks exectly what we want
# we have our linear regression formula in constraints
# we have our topic constraint

4 Set Declarations
    a_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'AG.LND.EL5M.ZS', 'AG.YLD.CREL.KG', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.CLC.MDAT.ZS', 'EN.POP.EL5M.ZS', 'EN.URB.MCTY.TL.ZS', 'ER.H2O.FWTL.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SH.STA.ACSN', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activation_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'AG.LND.EL5M.ZS', 'AG.YLD.CREL.KG', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.CLC.MDAT.ZS', 'EN.POP.EL5M.ZS', 'EN.URB.MCTY.TL.ZS', 'ER.H2O.FWTL.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.

## Solve!

In [ ]:
# solve it
SolverFactory('bonmin', executable='/content/bonmin').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 54
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 7.890924453735352
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [ ]:
model1_result = model.cost()
print('Sum of variance:',model.cost())

Sum of variance: 1677266814.6155179


## Compare your results to actual

In [ ]:
for country in countries:
  print(country,' (Predict): ', model.y[country]()) #out models predicted outcome
  print(country,' (Actual): ', float(df['average'][(df['Series name'] == 'CO2 emissions, total (KtCO2)') & (df['Country code'] == country)])) # the actual value
  print('-'*50)

BRA  (Predict):  309023.5072242647
BRA  (Actual):  304796.215
--------------------------------------------------
CHE  (Predict):  60680.57654746303
CHE  (Actual):  41050.13500000001
--------------------------------------------------
CHL  (Predict):  19886.335516549178
CHL  (Actual):  53241.752
--------------------------------------------------
CZE  (Predict):  122922.91339230722
CZE  (Actual):  125148.88594117649
--------------------------------------------------
DEU  (Predict):  812361.8229072299
DEU  (Actual):  817294.591
--------------------------------------------------
DNK  (Predict):  65422.914498700535
DNK  (Actual):  53926.323
--------------------------------------------------


## Which variables made it into your model?

We can see that any activated variable (1) is included in the model. They are:
* AG.YLD.CREL.KG
* EN.CLC.HPPT.MM
* SP.POP.TOTL
* SP.URB.TOTL

In [ ]:
for indicator in indicators:
  print(indicator,' : ',model.activation[indicator]())

AG.LND.EL5M.ZS  :  0.0
AG.YLD.CREL.KG  :  1.0
BX.KLT.DINV.WD.GD.ZS  :  0.0
EN.CLC.HPPT.MM  :  1.0
EN.CLC.MDAT.ZS  :  0.0
EN.POP.EL5M.ZS  :  0.0
EN.URB.MCTY.TL.ZS  :  0.0
ER.H2O.FWTL.ZS  :  0.0
ER.LND.PTLD.ZS  :  0.0
IC.BUS.EASE.XQ  :  0.0
IS.ROD.PAVE.ZS  :  0.0
NY.GDP.MKTP.CD  :  0.0
NY.GNP.PCAP.CD  :  0.0
SE.ENR.PRSC.FM.ZS  :  0.0
SE.PRM.CMPT.ZS  :  0.0
SH.DYN.MORT  :  0.0
SH.H2O.SAFE.ZS  :  0.0
SH.MED.NUMW.P3  :  0.0
SH.MED.PHYS.ZS  :  0.0
SH.STA.ACSN  :  0.0
SP.POP.GROW  :  0.0
SP.POP.TOTL  :  1.0
SP.URB.GROW  :  0.0
SP.URB.TOTL  :  1.0


# Experimenting with Different Parameters


* Test the model preforms with 3 or 5 predictors. 
* Test the model without the constraint for 'one from each group'.

* Run it 10 times... and change intial conditions... explore any differences in results.

## 3 predictors
Because we have a constraint that we have to choose at least one variable in each topic and we have 4 topic totally, we can't choose only 3 predictors. Otherwise the constraints will conflict to each other.

## 5 predictors

### Concrete Model

In [ ]:
# declare the model
model = ConcreteModel()

### Decision Variables
You should have multiple variables here
* `a` are the coefficients on the terms in your model - one `a` for each variable. Use `Var(indicators, domain=XYZ, bounds=ABC)` to do this.
* `activation` is whether or not the variable turns on (1) or not (0) - one `activation` for each variable in the model.  Use `Var(indicators, domain=XYZ, bounds=ABC)` to do this.
* `y` is a decision variable which enforces the linear form of the model (these are your intermediate predictions for each country, so you should only have 6).  Use `Var(countries, domain=XYZ, bounds=ABC, initialize=Z)` to do this.

In [ ]:
# declare the decision variables
model.a = Var(indicators, domain=Reals, bounds = (-100, 100)) # coefficients
model.y = Var(countries, domain=NonNegativeReals, initialize=0) # predict target variable
model.activation = Var(indicators, domain = Binary) # activation variables


### Constraints

In [ ]:
# Constraints
model.constraints = ConstraintList()

In [ ]:
# constraint for the topics
# declare variables
topic1=0
topic2=0
topic3=0
topic4=0


for indicator in indicator_dict[(list(indicator_dict.keys())[0])]:
  topic1 += model.activation[indicator]

for indicator in indicator_dict[(list(indicator_dict.keys())[1])]:
  topic2 += model.activation[indicator]

for indicator in indicator_dict[(list(indicator_dict.keys())[2])]:
  topic3 += model.activation[indicator]

for indicator in indicator_dict[(list(indicator_dict.keys())[3])]:
  topic4 += model.activation[indicator]

# constraint for the perdict variables
# declare variables
Predict_BRA=0
Predict_CHE=0
Predict_CHL=0
Predict_CZE=0
Predict_DEU=0
Predict_DNK=0

# the equation for the perdict variables
for indicator in indicators:
 Predict_BRA += model.a[indicator]*model.activation[indicator]*var_dict['BRA',indicator] # here is the perdicting formula y = A1N1 + A2N2 + .... AnNn

for indicator in indicators:
  Predict_CHE += model.a[indicator]*model.activation[indicator]*var_dict['CHE',indicator]

for indicator in indicators:
  Predict_CHL += model.a[indicator]*model.activation[indicator]*var_dict['CHL',indicator]

for indicator in indicators:
  Predict_CZE += model.a[indicator]*model.activation[indicator]*var_dict['CZE',indicator]

for indicator in indicators:
  Predict_DEU += model.a[indicator]*model.activation[indicator]*var_dict['DEU',indicator]

for indicator in indicators:
  Predict_DNK += model.a[indicator]*model.activation[indicator]*var_dict['DNK',indicator]

# add the constraints
model.constraints.add(topic1 >= 1) # we should at least choose one indicator for each topic
model.constraints.add(topic2 >= 1)
model.constraints.add(topic3 >= 1)
model.constraints.add(topic4 >= 1)
model.constraints.add(topic1 + topic2 + topic3 + topic4 <=5) # our indicators should be less or equal to 5

model.constraints.add(Predict_BRA == model.y['BRA']) # these are the equations to bulld linear regression models for each country
model.constraints.add(Predict_CHE == model.y['CHE'])
model.constraints.add(Predict_CHL == model.y['CHL'])
model.constraints.add(Predict_CZE == model.y['CZE'])
model.constraints.add(Predict_DEU == model.y['DEU'])
model.constraints.add(Predict_DNK == model.y['DNK'])

In [ ]:
# check the actual number of our target variable, we will need that in the model
df[df['Series name'] == 'CO2 emissions, total (KtCO2)']

,Country code,Country name,Series code,Series name,SCALE,Decimals,Scale,Order,Topic,Definition,Source,average
1657,BRA,Brazil,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",304796.215000
1664,CHE,Switzerland,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",41050.135000
1666,CHL,Chile,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",53241.752000
1680,CZE,Czech Republic,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",125148.885941
1681,DEU,Germany,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",817294.591000
1684,DNK,Denmark,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",53926.323000


In [ ]:
# Objective
obj_expr = 0
for country in countries:
    obj_expr += (float(df['average'][(df['Series name'] == 'CO2 emissions, total (KtCO2)') & (df['Country code'] == country)]) - model.y[country]) ** 2
    

model.cost = Objective(
    expr = obj_expr, 
    sense = minimize)

### Pretty print
... and check your constraints!

In [ ]:
model.pprint()
# check the result, we have a for each indicator, activation for each indicator
# objective function looks exectly what we want
# we have our linear regression formula in constraints
# we have our topic constraint

4 Set Declarations
    a_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'AG.LND.EL5M.ZS', 'AG.YLD.CREL.KG', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.CLC.MDAT.ZS', 'EN.POP.EL5M.ZS', 'EN.URB.MCTY.TL.ZS', 'ER.H2O.FWTL.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SH.STA.ACSN', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activation_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'AG.LND.EL5M.ZS', 'AG.YLD.CREL.KG', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.CLC.MDAT.ZS', 'EN.POP.EL5M.ZS', 'EN.URB.MCTY.TL.ZS', 'ER.H2O.FWTL.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.

### Solve!

In [ ]:
# solve it
SolverFactory('bonmin', executable='/content/bonmin').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 54
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 3.8097481727600098
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [ ]:
model2_result = model.cost()
print('Sum of variance:',model.cost())

Sum of variance: 361318956.5757225


### Compare your results to actual

In [ ]:
for country in countries:
  print(country,' (Predict): ', model.y[country]())
  print(country,' (Actual): ', float(df['average'][(df['Series name'] == 'CO2 emissions, total (KtCO2)') & (df['Country code'] == country)]))
  print('-'*50)

BRA  (Predict):  306287.0863038748
BRA  (Actual):  304796.215
--------------------------------------------------
CHE  (Predict):  41663.76699632882
CHE  (Actual):  41050.13500000001
--------------------------------------------------
CHL  (Predict):  42282.58549342542
CHL  (Actual):  53241.752
--------------------------------------------------
CZE  (Predict):  140287.3580384455
CZE  (Actual):  125148.88594117649
--------------------------------------------------
DEU  (Predict):  814240.8312740622
DEU  (Actual):  817294.591
--------------------------------------------------
DNK  (Predict):  53583.39635812426
DNK  (Actual):  53926.323
--------------------------------------------------


### Which variables made it into your model?

In [ ]:
for indicator in indicators:
  print(indicator,' : ',model.activation[indicator]())

AG.LND.EL5M.ZS  :  0.0
AG.YLD.CREL.KG  :  1.0
BX.KLT.DINV.WD.GD.ZS  :  0.0
EN.CLC.HPPT.MM  :  1.0
EN.CLC.MDAT.ZS  :  0.0
EN.POP.EL5M.ZS  :  0.0
EN.URB.MCTY.TL.ZS  :  0.0
ER.H2O.FWTL.ZS  :  0.0
ER.LND.PTLD.ZS  :  0.0
IC.BUS.EASE.XQ  :  0.0
IS.ROD.PAVE.ZS  :  0.0
NY.GDP.MKTP.CD  :  0.0
NY.GNP.PCAP.CD  :  1.0
SE.ENR.PRSC.FM.ZS  :  0.0
SE.PRM.CMPT.ZS  :  0.0
SH.DYN.MORT  :  0.0
SH.H2O.SAFE.ZS  :  0.0
SH.MED.NUMW.P3  :  0.0
SH.MED.PHYS.ZS  :  0.0
SH.STA.ACSN  :  0.0
SP.POP.GROW  :  0.0
SP.POP.TOTL  :  1.0
SP.URB.GROW  :  0.0
SP.URB.TOTL  :  1.0


## Get rid of the constraint for 'one from each group'

### Concrete Model

In [ ]:
# declare the model
model = ConcreteModel()

### Decision Variables
You should have multiple variables here
* `a` are the coefficients on the terms in your model - one `a` for each variable. Use `Var(indicators, domain=XYZ, bounds=ABC)` to do this.
* `activation` is whether or not the variable turns on (1) or not (0) - one `activation` for each variable in the model.  Use `Var(indicators, domain=XYZ, bounds=ABC)` to do this.
* `y` is a decision variable which enforces the linear form of the model (these are your intermediate predictions for each country, so you should only have 6).  Use `Var(countries, domain=XYZ, bounds=ABC, initialize=Z)` to do this.

In [ ]:
# declare the decision variables
model.a = Var(indicators, domain=Reals, bounds = (-100, 100)) # coefficients
model.y = Var(countries, domain=NonNegativeReals, initialize=0) # predict target variable
model.activation = Var(indicators, domain = Binary) # activation variables


### Constraints

In [ ]:
# Constraints
model.constraints = ConstraintList()

In [ ]:
# constraint for the topics
# declare variables
topic1=0
topic2=0
topic3=0
topic4=0


for indicator in indicator_dict[(list(indicator_dict.keys())[0])]:
  topic1 += model.activation[indicator]

for indicator in indicator_dict[(list(indicator_dict.keys())[1])]:
  topic2 += model.activation[indicator]

for indicator in indicator_dict[(list(indicator_dict.keys())[2])]:
  topic3 += model.activation[indicator]

for indicator in indicator_dict[(list(indicator_dict.keys())[3])]:
  topic4 += model.activation[indicator]

# constraint for the perdict variables
# declare variables
Predict_BRA=0
Predict_CHE=0
Predict_CHL=0
Predict_CZE=0
Predict_DEU=0
Predict_DNK=0

# the equation for the perdict variables
for indicator in indicators:
 Predict_BRA += model.a[indicator]*model.activation[indicator]*var_dict['BRA',indicator] # here is the perdicting formula y = A1N1 + A2N2 + .... AnNn

for indicator in indicators:
  Predict_CHE += model.a[indicator]*model.activation[indicator]*var_dict['CHE',indicator]

for indicator in indicators:
  Predict_CHL += model.a[indicator]*model.activation[indicator]*var_dict['CHL',indicator]

for indicator in indicators:
  Predict_CZE += model.a[indicator]*model.activation[indicator]*var_dict['CZE',indicator]

for indicator in indicators:
  Predict_DEU += model.a[indicator]*model.activation[indicator]*var_dict['DEU',indicator]

for indicator in indicators:
  Predict_DNK += model.a[indicator]*model.activation[indicator]*var_dict['DNK',indicator]

# add the constraints
model.constraints.add(topic1 + topic2 + topic3 + topic4 <=4) # our indicators should be less or equal to 4 

model.constraints.add(Predict_BRA == model.y['BRA']) # these are the equations to bulld linear regression models for each country
model.constraints.add(Predict_CHE == model.y['CHE'])
model.constraints.add(Predict_CHL == model.y['CHL'])
model.constraints.add(Predict_CZE == model.y['CZE'])
model.constraints.add(Predict_DEU == model.y['DEU'])
model.constraints.add(Predict_DNK == model.y['DNK'])

In [ ]:
# check the actual number of our target variable, we will need that in the model
df[df['Series name'] == 'CO2 emissions, total (KtCO2)']

,Country code,Country name,Series code,Series name,SCALE,Decimals,Scale,Order,Topic,Definition,Source,average
1657,BRA,Brazil,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",304796.215000
1664,CHE,Switzerland,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",41050.135000
1666,CHL,Chile,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",53241.752000
1680,CZE,Czech Republic,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",125148.885941
1681,DEU,Germany,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",817294.591000
1684,DNK,Denmark,EN.ATM.CO2E.KT,"CO2 emissions, total (KtCO2)",0,1,0,43,GHG emissions and energy use,Carbon dioxide emissions are those stemming fr...,"Carbon Dioxide Information Analysis Center, En...",53926.323000


In [ ]:
# Objective
obj_expr = 0                  #Dummy variable set
for country in countries:
    #the objective function is the sum of squared differences of the actual value of each country and the predicted value
    obj_expr += (float(df['average'][(df['Series name'] == 'CO2 emissions, total (KtCO2)') & (df['Country code'] == country)]) - model.y[country]) ** 2 
    
# we will minimize the sum of squared differences to find the best model
model.cost = Objective(
    expr = obj_expr, 
    sense = minimize)

### Pretty print
... and check your constraints!

In [ ]:
model.pprint()
# check the result, we have a for each indicator, activation for each indicator
# objective function looks exectly what we want
# we have our linear regression formula in constraint
# we have our topic constraint

4 Set Declarations
    a_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'AG.LND.EL5M.ZS', 'AG.YLD.CREL.KG', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.CLC.MDAT.ZS', 'EN.POP.EL5M.ZS', 'EN.URB.MCTY.TL.ZS', 'ER.H2O.FWTL.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SH.STA.ACSN', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activation_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'AG.LND.EL5M.ZS', 'AG.YLD.CREL.KG', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.CLC.MDAT.ZS', 'EN.POP.EL5M.ZS', 'EN.URB.MCTY.TL.ZS', 'ER.H2O.FWTL.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.

### Solve!

In [ ]:
# solve it
SolverFactory('bonmin', executable='/content/bonmin').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 54
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 3.067517042160034
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [ ]:
model3_result = model.cost()
print('Sum of variance:',model.cost())

Sum of variance: 1677266814.7504923


In [ ]:
print('Original model:',model1_result)
print('5 predictors model:',model2_result)
print('remove one constraint model:',model3_result)

Original model: 1677266814.6155179
5 predictors model: 361318956.5757225
remove one constraint model: 1677266814.7504923


***From the result we can see that the model is getting better with more predictors. The model doesn't change when we get rid of the constraint for 'one from each group', which means this constraint is not a binding constraint.***

### Compare your results to actual

In [ ]:
for country in countries:
  print(country,' (Predict): ', model.y[country]())
  print(country,' (Actual): ', float(df['average'][(df['Series name'] == 'CO2 emissions, total (KtCO2)') & (df['Country code'] == country)]))
  print('-'*50)

BRA  (Predict):  309023.50722446863
BRA  (Actual):  304796.215
--------------------------------------------------
CHE  (Predict):  60680.57654828425
CHE  (Actual):  41050.13500000001
--------------------------------------------------
CHL  (Predict):  19886.33551506359
CHL  (Actual):  53241.752
--------------------------------------------------
CZE  (Predict):  122922.9133930002
CZE  (Actual):  125148.88594117649
--------------------------------------------------
DEU  (Predict):  812361.8229069097
DEU  (Actual):  817294.591
--------------------------------------------------
DNK  (Predict):  65422.91449878014
DNK  (Actual):  53926.323
--------------------------------------------------


### Which variables made it into your model?

In [ ]:
for indicator in indicators:
  print(indicator,' : ',model.activation[indicator]())

AG.LND.EL5M.ZS  :  0.0
AG.YLD.CREL.KG  :  1.0
BX.KLT.DINV.WD.GD.ZS  :  0.0
EN.CLC.HPPT.MM  :  1.0
EN.CLC.MDAT.ZS  :  0.0
EN.POP.EL5M.ZS  :  0.0
EN.URB.MCTY.TL.ZS  :  0.0
ER.H2O.FWTL.ZS  :  0.0
ER.LND.PTLD.ZS  :  0.0
IC.BUS.EASE.XQ  :  0.0
IS.ROD.PAVE.ZS  :  0.0
NY.GDP.MKTP.CD  :  0.0
NY.GNP.PCAP.CD  :  0.0
SE.ENR.PRSC.FM.ZS  :  0.0
SE.PRM.CMPT.ZS  :  0.0
SH.DYN.MORT  :  0.0
SH.H2O.SAFE.ZS  :  0.0
SH.MED.NUMW.P3  :  0.0
SH.MED.PHYS.ZS  :  0.0
SH.STA.ACSN  :  0.0
SP.POP.GROW  :  0.0
SP.POP.TOTL  :  1.0
SP.URB.GROW  :  0.0
SP.URB.TOTL  :  1.0


#Conclusion

Conclusion: Based on the result, if we slightly add more predictors, the result may be more accurate. But if we subset a test dataset from the training dataset, we may see that the model is going to overfit. Every time we change the constraints, the result will change. The thing is, the constraints are helping us to find the optimal solution based on what we have. So we recommend avoiding overfitting while trying to improve accuracy.